**Radioactive decay chain Bismuth-Lead**

\begin{equation}
_{83}^{210}\text{Bi} \;\; ^{\beta^{-}} \rightarrow \; _{84}^{210}\text{Po} \;\; ^{\alpha}\rightarrow \; _{82}^{206}\text{Pb}
\end{equation}

System of Ordinary Differential Equations (ODE)

\begin{aligned}
\frac{d \text{Bi}}{d t} &= -\lambda_{\text{Bi}}\text{Bi} \\
\frac{d \text{Po}}{d t} &= -\lambda_{\text{Po}}\text{Po} + \lambda_{\text{Bi}}\text{Bi} \\
\frac{d \text{Pb}}{d t} &= \lambda_{\text{Po}}\text{Po} \\
\end{aligned}

Analytical solution:

\begin{aligned}
\text{Bi}(t) &= \text{Bi}_{0}\exp(-\lambda_{Bi}t) \\
\text{Po}(t) &= \lambda_{\text{Bi}}\text{Bi}_{0}\left(\frac{\exp(-\lambda_{Bi}t)}{\lambda_{\text{Po}}-\lambda_{\text{Bi}}} + \frac{\exp(-\lambda_{Po}t)}{\lambda_{\text{Bi}}-\lambda_{\text{Po}}} \right) \\
\text{Pb}(t) &= \text{Bi}_{0}\left(1 - \frac{\lambda_{\text{Po}}\exp(-\lambda_{Bi}t)}{\lambda_{\text{Po}}-\lambda_{\text{Bi}}} - \frac{\lambda_{\text{Bi}}\exp(-\lambda_{Po}t)}{\lambda_{\text{Bi}}-\lambda_{\text{Po}}} \right)
\end{aligned}

In [ ]:
# Import the module

import sympy as sp
import numpy as np

# Define the variables and the functions

t = sp.Symbol('t',nonnegative = True, Real = True)
Bi0 = sp.Symbol('Bi_0', nonnegative = True, Real = True)
LambdaBi = sp.Symbol('lambda_B', positive = True, Real = True)
LambdaPo = sp.Symbol('lambda_P',positive = True, Real = True)

nbi, npo, npb = sp.symbols('Bi Po Pb', cls = sp.Function)

# Define the initial conditions

IC = {
    nbi(0):Bi0,
    npo(0):0,
    npb(0):0
}

# Write the system of equations

System = [
    sp.Eq(nbi(t).diff(t),-LambdaBi*nbi(t)),
    sp.Eq(npo(t).diff(t),-LambdaPo*npo(t)+LambdaBi*nbi(t)),
    sp.Eq(npb(t).diff(t),LambdaPo*npo(t))
]

In [ ]:
# We have a system of ODE, in the documentation look for the suitable solver

In [ ]:
# Solve the system of equations

from sympy.solvers.ode import dsolve
from sympy import latex

SolutionIC = sp.dsolve(System, ics = IC) # If I want to specify the Initial conditions
Solution = sp.dsolve(System)

print(Solution[0])
print(Solution[1])
print(Solution[2])

In [ ]:
# Display the equation for Bi

SolutionIC[0]

In [ ]:
# Display the equation for Po

SolutionIC[1]

In [ ]:
# Display the equation for Pb

SolutionIC[2]

In [ ]:
# Define the numerical quantities and evaluate

TauBi = sp.Float(5.012) #days
TauPo = sp.Float(138.376) #days
BiInitial = sp.Float(2E-10)

LambdaBiNum = np.log(2)/TauBi #sp.log(2)/TauBi
LambdaPoNum = np.log(2)/TauPo

# Substitute the values

BiConcentration = SolutionIC[0].rhs.subs({
    Bi0:BiInitial,
    LambdaBi:LambdaBiNum
})

PoConcentration = SolutionIC[1].rhs.subs({
    Bi0:BiInitial,
    LambdaBi:LambdaBiNum,
    LambdaPo:LambdaPoNum
})

PbConcentration = SolutionIC[2].rhs.subs({
    Bi0:BiInitial,
    LambdaBi:LambdaBiNum,
    LambdaPo:LambdaPoNum
})

print(f'Bi concentration: {BiConcentration}')
print(f'Po concentration: {PoConcentration}')
print(f'Pb concentration: {PbConcentration}')

In [ ]:
# Plot (use matplotlib)

import matplotlib.pyplot as plt

BiLambdified = sp.lambdify(t, BiConcentration, 'numpy')
PoLambdified = sp.lambdify(t, PoConcentration, 'numpy')
PbLambdified = sp.lambdify(t, PbConcentration, 'numpy')

Time = np.arange(0,30,0.01)

plt.plot(Time,BiLambdified(Time),label = 'Bi')
plt.plot(Time,PoLambdified(Time),label = 'Po')
plt.plot(Time,PbLambdified(Time),label = 'Pb')
plt.title('Bismuth - Lead decay chain')
plt.xlabel('Time (days)')
plt.ylabel('Concentration')
plt.legend(loc = 'best')
plt.grid()

Asymptotic behaviour (analytical):

\begin{aligned}
\text{lim}_{t \rightarrow +\infty} \text{Bi}(t) &= \text{Bi}_{0}\exp(-\lambda_{\text{Bi}}(+\infty)) = 0 \\
\text{lim}_{t \rightarrow +\infty} \text{Po}(t) &= \lambda_{\text{Bi}}\text{Bi}_{0}\left(\frac{\exp(-\lambda_{Bi}(+\infty))}{\lambda_{\text{Po}}-\lambda_{\text{Bi}}} + \frac{\exp(-\lambda_{Po}(+\infty))}{\lambda_{\text{Bi}}-\lambda_{\text{Po}}} \right) = 0 \\
\text{lim}_{t \rightarrow +\infty} \text{Pb}(t) &= \text{Bi}_{0}\left(1 - \frac{\lambda_{\text{Po}}\exp(-\lambda_{Bi}(+\infty))}{\lambda_{\text{Po}}-\lambda_{\text{Bi}}} - \frac{\lambda_{\text{Bi}}\exp(-\lambda_{Po}(+\infty))}{\lambda_{\text{Bi}}-\lambda_{\text{Po}}} \right) = \text{Bi}_{0}
\end{aligned}

In [ ]:
# Compute the asymptotic behaviour using Sympy
# Warning: for limits, you should specify if your variables are negative or positive!
# In this case, the limit will depend on the sign of lambdaBi and lambdaPo

BiAsympt = sp.limit(SolutionIC[0].rhs,t,sp.oo)
PoAsympt = sp.limit(SolutionIC[1].rhs,t,sp.oo)
PbAsympt = sp.limit(SolutionIC[2].rhs,t,sp.oo)

print(f'Asymptotic Bismuth = {BiAsympt}')
print(f'Asymptotic Polonium = {PoAsympt}')
print(f'Asymptotic Lead = {PbAsympt}')

In [ ]:
# Now compute the concentration after 1 year

Bi1Year = BiConcentration.evalf(subs = {t:365})
Po1Year = PoConcentration.evalf(subs = {t:365})
Pb1Year = PbConcentration.evalf(subs = {t:365})

print(f'Bi concentration after 1 year = {Bi1Year}')